In [0]:
import os
import zipfile

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
local_zip='/content/pdd.zip'


In [0]:
zip_ref=zipfile.ZipFile(local_zip,'r')


In [0]:
zip_ref.extractall('/content/pdd')

In [0]:
local_zip='/content/pdd.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/content/pdd')

In [0]:
os.listdir('/content/pdd/pd/Plant Disease')

['images (44).jfif',
 'download (9).jfif',
 'images (19).jfif',
 'download (10).jfif',
 'download (12).jfif',
 'download (6).jfif',
 'images (7).jfif',
 'download (5).jfif',
 'images (18).jfif',
 'images (20).jfif',
 'images (17).jfif',
 'download (7).jfif',
 'images (5).jfif',
 'images (4).jfif',
 'images (16).jfif',
 'images (13).jfif',
 'images (11).jfif',
 'images (15).jfif',
 'download (3).jfif',
 'images (1).jfif',
 'images (12).jfif',
 'images.jfif',
 'images (6).jfif',
 'images (8).jfif',
 'download (11).jfif',
 'download (8).jfif',
 'images (9).jfif',
 'download (2).jfif',
 'images (14).jfif',
 'download (4).jfif',
 'download (1).jfif',
 'images (2).jfif',
 'Thumbs.db',
 'images (3).jfif']

In [0]:
train_plant_dir = os.path.join('/content/pdd/pd/plant')


In [0]:
train_disease_dir = os.path.join('/content/pdd/pd/Plant Disease')

In [0]:
train_plant_names = os.listdir(train_plant_dir)
print(train_plant_names[:10])


['sdf.jfif', 'fd (5).jfif', 'fd (29).jfif', 'fd (7).jfif', 'fd (8).jfif', 'fd (14).jfif', 'fd (12).jfif', 'd.jfif', 'fd (28).jfif', 'fd (22).jfif']


In [0]:
train_disease_names = os.listdir(train_disease_dir)
print(train_disease_names[:10])


['images (44).jfif', 'download (9).jfif', 'images (19).jfif', 'download (10).jfif', 'download (12).jfif', 'download (6).jfif', 'images (7).jfif', 'download (5).jfif', 'images (18).jfif', 'images (20).jfif']


In [0]:
print('total training plant images:', len(os.listdir(train_plant_dir)))
print('total training disease images:', len(os.listdir(train_disease_dir)))

total training plant images: 34
total training disease images: 34


In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [0]:
# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

In [0]:
pic_index = 0

In [0]:
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

<Figure size 1152x1152 with 0 Axes>

In [0]:
pic_index += 8
next_plantpix = [os.path.join(train_plant_dir, fname) 
                for fname in train_plant_names[pic_index-8:pic_index]]
next_disease_pix = [os.path.join(train_disease_dir, fname) 
                for fname in train_disease_names[pic_index-8:pic_index]]

In [0]:
import tensorflow as tf


In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 33, 33, 64)       

In [0]:
from tensorflow.keras.optimizers import RMSprop

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255)


In [0]:
train_generator = train_datagen.flow_from_directory(
        '/content/pdd/pd',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

In [0]:
his=model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)



Epoch 1/15
8/8 [==============================] - 86s 11s/step - loss: 0.0822 - acc: 1.0000
Epoch 2/15
8/8 [==============================] - 80s 10s/step - loss: 1.0026e-06 - acc: 1.0000
Epoch 3/15
8/8 [==============================] - 80s 10s/step - loss: 1.5182e-06 - acc: 1.0000
Epoch 4/15
8/8 [==============================] - 80s 10s/step - loss: 1.3744e-07 - acc: 1.0000
Epoch 5/15
8/8 [==============================] - 80s 10s/step - loss: 9.3813e-08 - acc: 1.0000
Epoch 6/15
8/8 [==============================] - 80s 10s/step - loss: 4.2957e-08 - acc: 1.0000
Epoch 7/15
8/8 [==============================] - 80s 10s/step - loss: 5.3388e-08 - acc: 1.0000
Epoch 8/15
8/8 [==============================] - 80s 10s/step - loss: 4.7215e-08 - acc: 1.0000
Epoch 9/15
8/8 [==============================] - 80s 10s/step - loss: 1.6527e-08 - acc: 1.0000
Epoch 10/15
8/8 [==============================] - 80s 10s/step - loss: 2.6757e-08 - acc: 1.0000
Epoch 11/15
8/8 [==========================

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

Using TensorFlow backend.


In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  plt.imshow(img,cmap='Greys')
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is braintumour")
  else:
    print(fn + " is not a braintumout")


Saving sdf.jfif to sdf.jfif
[0.9999995]
sdf.jfif is Normalleaf
